In [24]:
if not os.path.isfile("estrutura_ud.py"):
    ! wget https://raw.githubusercontent.com/alvelvis/ACDC-UD/master/estrutura_ud.py
import estrutura_ud

In [33]:
file = "../Petroles_v2/Petroles_2.conllu"
corpus = estrutura_ud.Corpus()
corpus.load(file)
for sentence in corpus.sentences.values():
    for token in sentence.tokens:
        if token.lemma == "ser" and token.upos == "AUX" and token.deprel == "aux:pass":
            token.head_token.feats = "|".join(sorted([x for x in token.head_token.feats.split("|") if x != "Voice=Pass"] + ["Voice=Pass"]))
        if token.lemma != "ser" and token.upos == "AUX" and token.deprel == "aux:pass":
            if token.lemma == "estar" and token.head_token.upos != "VERB":
                token.deprel = "cop"
            else:
                token.deprel = "aux"
    for token in sentence.tokens:
        if 'Voice=Pass' in token.feats:
            for _token in sentence.tokens:
                if _token.dephead == token.id and _token.lemma == "ser" and token.upos == "AUX":
                    _token.deprel = "aux:pass"
                if _token.dephead == token.id and _token.deprel == "nsubj":
                    _token.deprel = "nsubj:pass"
                
    for token in sentence.tokens:
        if not 'Voice=Pass' in token.head_token.feats and (token.deprel in ["nsubj:pass", "obl:agent", "aux:pass"]):
            token.deprel = token.deprel.split(":")[0]

        if token.deprel == "cop" and token.head_token.upos == "VERB":
            if token.lemma == "estar":
                token.deprel = "aux"
            if token.lemma == "ser":
                token.deprel = token.head_token.deprel
                token.dephead = token.head_token.dephead
                token.head_token.deprel = "ccomp"
                token.head_token.dephead = token.id
                token.upos = "VERB"
                for _token in sentence.tokens:
                    if _token.dephead == token.head_token.id:
                        if _token.deprel in ["nsubj", "punct"]:
                            _token.dephead = token.id


corpus.save("teste.conllu")
os.system("meld --diff {} teste.conllu".format(file))
os.remove("teste.conllu")

build: 0.8006019592285156